# Federal University of Campina Grande (UFCG)
# Author: Rayana Souza Rocha
# Available at: https://github.com/rayanarocha/machinelearning/tree/590e295d554c0fffdcf11093abd7dbac74e5ce91/previs%C3%A3o-de-vota%C3%A7%C3%A3o-de-deputados

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr


%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [ ]:
all_data = pd.read_csv("../previsão-de-votação-de-deputados/eleicoes_2014.csv")

In [ ]:
all_data.info()

In [ ]:
all_data.describe(include=['object'])

In [ ]:
all_data.votos.isnull().sum()

In [ ]:
msk = np.random.rand(len(all_data)) < 0.8 # cria uma mascara com as posicoes
train = all_data[msk]
test = all_data[~msk]

In [ ]:
train.head(5)

In [ ]:
test.head(5)

In [ ]:
train.info()

Determinando números máximos de cada coluna

In [ ]:
train.max()

Determinando números mínimos de cada coluna

In [ ]:
train.min()

Normalizando o modelos

In [ ]:
from sklearn.preprocessing import StandardScaler
normalizador = StandardScaler()

In [ ]:
train[["ano","sq_candidato","qtd_doacoes","qtd_doadores","total_receita","media_receita","recursos de outros candidatos/comitês","recursos de pessoas físicas","recursos de pessoas jurídicas","recursos próprios","qtd_despesas","qtd_fornecedores","total_despesa","media_despesa","votos"]] = normalizador.fit_transform(train[["ano","sq_candidato","qtd_doacoes","qtd_doadores","total_receita","media_receita","recursos de outros candidatos/comitês","recursos de pessoas físicas","recursos de pessoas jurídicas","recursos próprios","qtd_despesas","qtd_fornecedores","total_despesa","media_despesa","votos"]])

In [ ]:
test[["ano","sq_candidato","qtd_doacoes","qtd_doadores","total_receita","media_receita","recursos de outros candidatos/comitês","recursos de pessoas físicas","recursos de pessoas jurídicas","recursos próprios","qtd_despesas","qtd_fornecedores","total_despesa","media_despesa","votos"]] = normalizador.fit_transform(test[["ano","sq_candidato","qtd_doacoes","qtd_doadores","total_receita","media_receita","recursos de outros candidatos/comitês","recursos de pessoas físicas","recursos de pessoas jurídicas","recursos próprios","qtd_despesas","qtd_fornecedores","total_despesa","media_despesa","votos"]])

In [ ]:
train.head(5)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
votos = pd.DataFrame({"votos":train["votos"], "log(price + 1)":np.log1p(train["votos"])})
votos.hist()

In [ ]:
#log transform the target:
train["votos"] = np.log1p(train["votos"])

#log transform skewed numeric features:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

In [ ]:
all_data = pd.get_dummies(all_data)

In [ ]:
#filling NA's with the mean of the column:
all_data = all_data.fillna(all_data.mean())

In [ ]:
#creating matrices for sklearn:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.votos

## Models

Now we are going to use regularized linear regression models from the scikit learn module. I'm going to try both l_1(Lasso) and l_2(Ridge) regularization. I'll also define a function that returns the cross-validation rmse error so we can evaluate our models and pick the best tuning par

In [ ]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

In [ ]:
model_ridge = Ridge()

The main tuning parameter for the Ridge model is alpha - a regularization parameter that measures how flexible our model is. The higher the regularization the less prone our model will be to overfit. However it will also lose flexibility and might not capture all of the signal in the data.

In [ ]:
alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]

In [ ]:
cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge.plot(title = "Validation - Just Do It")
plt.xlabel("alpha")
plt.ylabel("rmse")

Note the U-ish shaped curve above. When alpha is too large the regularization is too strong and the model cannot capture all the complexities in the data. If however we let the model be too flexible (alpha small) the model begins to overfit. A value of alpha = 10 is about right based on the plot above.

In [ ]:
cv_ridge.min()

So for the Ridge regression we get a rmsle of about 0.127

Let' try out the Lasso model. We will do a slightly different approach here and use the built in Lasso CV to figure out the best alpha for us. For some reason the alphas in Lasso CV are really the inverse or the alphas in Ridge.

In [ ]:
model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]).fit(X_train, y)

In [ ]:
rmse_cv(model_lasso).mean()

Nice! The lasso performs even better so we'll just use this one to predict on the test set. Another neat thing about the Lasso is that it does feature selection for you - setting coefficients of features it deems unimportant to zero. Let's take a look at the coefficients:

In [ ]:
coef = pd.Series(model_lasso.coef_, index = X_train.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

Good job Lasso. One thing to note here however is that the features selected are not necessarily the "correct" ones - especially since there are a lot of collinear features in this dataset. One idea to try here is run Lasso a few times on boostrapped samples and see how stable the feature selection is.

We can also take a look directly at what the most important coefficients are:

In [ ]:
imp_coef = pd.concat([coef.sort_values().head(10),
                     coef.sort_values().tail(10)])

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients in the Lasso Model")

The most important positive feature is GrLivArea - the above ground area by area square feet. This definitely sense. Then a few other location and quality features contributed positively. Some of the negative features make less sense and would be worth looking into more - it seems like they might come from unbalanced categorical variables.

Also note that unlike the feature importance you'd get from a random forest these are actual coefficients in your model - so you can say precisely why the predicted price is what it is. The only issue here is that we log_transformed both the target and the numeric features so the actual magnitudes are a bit hard to interpret.

In [ ]:
#let's look at the residuals as well:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds":model_lasso.predict(X_train), "true":y})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")

The residual plot looks pretty good.To wrap it up let's predict on the test set and submit on the leaderboard:

## Adding an xgboost model:

Let's add an xgboost model to our linear model to see if we can improve our score:

In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(X_train, label = y)
dtest = xgb.DMatrix(X_test)

params = {"max_depth":2, "eta":0.1}
model = xgb.cv(params, dtrain,  num_boost_round=500, early_stopping_rounds=100)

In [ ]:
model.loc[30:,["test-rmse-mean", "train-rmse-mean"]].plot()

In [ ]:
model_xgb = xgb.XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1) #the params were tuned using xgb.cv
model_xgb.fit(X_train, y)

In [ ]:
xgb_preds = np.expm1(model_xgb.predict(X_test))
lasso_preds = np.expm1(model_lasso.predict(X_test))

In [ ]:
predictions = pd.DataFrame({"xgb":xgb_preds, "lasso":lasso_preds})
predictions.plot(x = "xgb", y = "lasso", kind = "scatter")

Many times it makes sense to take a weighted average of uncorrelated results - this usually imporoves the score although in this case it doesn't help that much.

In [ ]:
preds = 0.7*lasso_preds + 0.3*xgb_preds

In [ ]:
solution = pd.DataFrame({"id":test.ano, "votos":preds})
solution.to_csv("ridge_sol.csv", index = False)

## Trying out keras?

Feedforward Neural Nets doesn't seem to work well at all...I wonder why.

In [ ]:
from keras.layers import Dense
from keras.models import Sequential
from keras.regularizers import l1
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
X_train = StandardScaler().fit_transform(X_train)

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y, random_state = 3)

In [ ]:
X_tr.shape

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from mlxtend.plotting import plot_decision_regions

In [ ]:
all_data['votos'] = pd.to_numeric(all_data['votos'])
dummies = pd.get_dummies(all_data)
x = np.array(dummies.drop(['votos'], 1))
y = np.array(dummies['votos'])

In [ ]:
from sklearn import model_selection

x_train_knn, x_test_knn, y_train_knn, y_test_knn = model_selection.train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
normalizacao_knn = StandardScaler()
x_train_knn = normalizacao_knn.fit_transform(x_train_knn)
x_test_knn = normalizacao_knn.fit_transform(x_test_knn)

In [ ]:
from sklearn import neighbors

model_clf = neighbors.KNeighborsRegressor(n_neighbors=80, metric='euclidean')

In [ ]:
model_clf.fit(x_train_knn, y_train_knn)

In [ ]:
accuracy = model_clf.score(x_test_knn, y_test_knn)
accuracy

In [ ]:
def knn_comparison(all_data, k):
    x = all_data[['sq_candidato','votos']].values
    y = all_data['votos'].astype(int).values
    knn = neighbors.KNeighborsRegressor(n_neighbors=k)
    knn.fit(x, y)
    plot_decision_regions(x, y, clf=knn)
    plt.xlabel('ssc_p')
    plt.ylabel('hsc_p')
    plt.title('K='+ str(k))
    plt.show()

In [ ]:
knn_comparison(all_data=dummies, k=80)

Os resultados obtidos para as regressões Ridge e Lasso forma iguais: 
Ridge: 0.48
Lasso: 0.48